In [1]:
import sys
sys.path.append("..")
from lib.kalman_gst import *  
from pygsti.modelpacks import smq1Q_XY as std

In [2]:
from pygsti.forwardsims import SimpleMatrixForwardSimulator

In [3]:
SAMPLES = 256

In [4]:
# setup the FOGI model
mdl_datagen = std.target_model('H+s')
basis1q = pygsti.baseobjs.Basis.cast('pp', 4)
gauge_basis = pygsti.baseobjs.CompleteElementaryErrorgenBasis(
                        basis1q, mdl_datagen.state_space, elementary_errorgen_types='HS')
mdl_datagen.setup_fogi(gauge_basis, None, None, reparameterize=True,
                     dependent_fogi_action='drop', include_spam=True)
target_model = mdl_datagen.copy()
filter_model = mdl_datagen.copy()

In [5]:
filter_model.fogi_errorgen_component_labels()

('H(X:0)_Gxpi2:0',
 'S(X:0)_Gxpi2:0',
 '(0.5 S(Y:0) + 0.5 S(Z:0))_Gxpi2:0',
 'H(Y:0)_Gypi2:0',
 '(0.5 S(X:0) + 0.5 S(Z:0))_Gypi2:0',
 'S(Y:0)_Gypi2:0',
 'ga(-H(Z:0))_Gypi2:0 - ga(-H(Z:0))_Gxpi2:0',
 'ga(H(Y:0))_rho0 - ga(H(Y:0))_Gxpi2:0',
 'ga(-H(Y:0))_Mdefault - ga(-H(Y:0))_Gxpi2:0',
 'ga(H(X:0))_rho0 - ga(H(X:0))_Gypi2:0',
 'ga(-H(X:0))_Mdefault - ga(-H(X:0))_Gypi2:0',
 'ga(-0.5 S(X:0) - 0.5 S(Y:0))_Mdefault - ga(-0.5 S(X:0) - 0.5 S(Y:0))_rho0')

In [6]:
filter_model.num_params

12

In [7]:
# print the model gate's gpindicies
print(filter_model[('rho0')].gpindices)
print(filter_model[('Mdefault')].gpindices)
for g in std.gates:
    print(filter_model[g].gpindices)

slice(0, 6, None)
slice(6, 12, None)
slice(12, 18, None)
slice(18, 24, None)


In [8]:
keys = ['Mdefault', 'rho0', 'Gxpi2', 'Gypi2']
gpidxs = { k : [] for k in keys } 
for key in keys:
    for idx, lbl in enumerate(filter_model.fogi_errorgen_component_labels()):
        if key in lbl:
            gpidxs[key].append(idx)

In [9]:
gpidxs

{'Mdefault': [8, 10, 11],
 'rho0': [7, 9, 11],
 'Gxpi2': [0, 1, 2, 6, 7, 8],
 'Gypi2': [3, 4, 5, 6, 9, 10]}

In [10]:
# reset the model gate's gpinidices
filter_model[('Gxpi2', 0)].set_gpindices(gpidxs['Gxpi2'], filter_model[('Gxpi2', 0)])
filter_model[('Gypi2', 0)].set_gpindices(gpidxs['Gypi2'], filter_model[('Gypi2', 0)])
filter_model[('Mdefault')].set_gpindices(gpidxs['Mdefault'], filter_model[('Mdefault')])
filter_model[('rho0')].set_gpindices(gpidxs['rho0'], filter_model[('rho0')])

In [11]:
# print the model gate's gpindicies
print(filter_model[('rho0')].gpindices)
print(filter_model[('Mdefault')].gpindices)
for g in std.gates:
    print(filter_model[g].gpindices)

[7, 9, 11]
[8, 10, 11]
[0, 1, 2, 6, 7, 8]
[3, 4, 5, 6, 9, 10]


In [12]:
std.fiducials()

[Circuit({}@(0)),
 Circuit(Gxpi2:0@(0)),
 Circuit(Gypi2:0@(0)),
 Circuit(Gxpi2:0Gxpi2:0@(0)),
 Circuit(Gxpi2:0Gxpi2:0Gxpi2:0@(0)),
 Circuit(Gypi2:0Gypi2:0Gypi2:0@(0))]

In [13]:
mfs = SimpleMatrixForwardSimulator(filter_model)
germs = std.germs()

dgerms = dict()
for g in germs:
    dgerms[g] = mfs.dproduct(g).reshape(filter_model.num_params, -1)

In [14]:
dgerms[germs[1]].shape

(12, 16)

In [15]:
filter_model.sim.dprobs(std.meas_fiducials()[1]+germs[1]+std.prep_fiducials()[0] )

OutcomeLabelDict([(('0',),
                   array([-2.29934694e-17,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                           1.25092819e-34, -2.81392211e-16,  4.49792587e-17,  6.52116990e-16,
                           2.28905593e-16, -3.06186218e-01, -9.18558654e-01, -5.00000000e-01])),
                  (('1',),
                   array([-2.29934694e-17,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                          -1.25092819e-34,  2.81392211e-16, -4.49792587e-17, -6.92323949e-16,
                          -3.49526471e-16,  3.06186218e-01,  9.18558654e-01,  5.00000000e-01]))])

In [17]:
mfs.dprobs(germs[0])

AssertionError: 

In [16]:
class ExtendedKalmanFilter():
    """
    An extended Kalman filter for gate-set tomography
    
    --- Parameters ---
    model: an underlying pygsti model
    num_params: number of parameters in the pygsti model
    P: current covariance matrix
    """
    def __init__(self, model, P0):
        self.model = model
        self.P = P0
        self.param_history = [self.model.to_vector()]
        self.covar_history = [self.P]
        
    def select_germ(self, germs, reps):
        cost_funs = []
        tfs = SimpleMatrixForwardSimulator(self.model)
        for g in germs:
            dgerms = tfs.dproduct(g*reps).reshape(filter_model.num_params, -1)
            print(dgerms.shape)
            cost_funs.append(np.trace(np.linalg.pinv(dgerms.T@self.P@dgerms)))
        return germs[np.argmax(cost_funs)]
    
    def select_fiducial(self, germ, reps, meas_fids, prep_fids):
        cost_funs = []
        cost_funs = np.zeros((len(meas_fids), len(prep_fids)))
        for idm, mf in enumerate(meas_fids):
            for idp, pf in enumerate(prep_fids):
                print(idm, idp)
                dprob = vector_from_outcomes(self.model.sim.dprobs(mf+germ*reps+pf), 2)
                print(dprob)
                cost_funs[mf, pf] = np.trace(np.linalg.pinv(dprob@self.P@dprob.T, 1e-6))
        idx_maxs = np.argmax(cost_funs)
        return (meas_fids[idx_maxs[0]], prep_fids[idx_maxs[1]])
        
        
    def update(self, circ, count_vec, clip_range=[-1,1], Q=None, R_additional=None, max_itr=1, itr_eps=1e-4):
        """
        Makes an exact update to the model
        where the jacobian is calculated as the current estimate
        
        --- Arguments ---
        circ: pygsti circuit used in the update
        count_vec: vector of observed counts
        clip_range: reasonable clipping range for the parameter update
        Q: state-space covariance 
        R_additional: additional measurement covariance
        max_itr: maximum number of iterations to the update
        itr_eps: epsilon for minimum difference to end iterated updates
        
        --- Returns --- 
        innovation: the prior innovation
        kgain: the Kalman gain
        """
        prior_covar = self.P
        prior_state = self.model.to_vector()
        hilbert_dims = 2**(circ.width)
        
        # find the predicted frequency for the circuit outcome under the model
        probs = self.model.probabilities(circ)
        p_model = vector_from_outcomes(probs, hilbert_dims)

        # calculate the observed frequency
        total_counts = sum(count_vec)
        observation = count_vec/total_counts

        # calculate jacobian
        jacob = matrix_from_jacob(self.model.sim.dprobs(circ), 2**circ.width)

        # calculate the covaraiance of the observation
        mean_frequency = ( count_vec+np.ones(len(count_vec)) )/( sum(count_vec)+len(count_vec) )
        R = (1/(sum(count_vec)+len(count_vec)+1))*categorical_covar(mean_frequency)

        # add any additional noise
        if R_additional is not None:
            R += R_additional
        if Q is None: 
            Q = 0*np.eye(self.model.num_params)

        # Kalman gain
        P = prior_covar + Q
        kgain = P@jacob.T@np.linalg.pinv(jacob@P@jacob.T + R, 1e-15)

        # Kalman update
        innovation = observation - p_model
        post_state = prior_state + kgain@innovation
        post_state = np.clip(post_state, clip_range[0], clip_range[1])
            

        
        # update class parameters
        self.P = (np.eye(self.model.num_params) - kgain@jacob)@P
        self.model.from_vector(post_state)
        
        self.param_history.append(post_state)
        self.covar_history.append(self.P)
        
        return innovation, kgain 

In [17]:
# add noise to the stochastic and hamiltonian parts of the FOGI rates
SEED = 3122
np.random.seed(SEED)

max_stochastic_error_rate = 0.001
hamiltonian_error_var = 0.05
ar = mdl_datagen.fogi_errorgen_components_array(include_fogv=False, normalized_elem_gens=True)


# add hamiltonian noise
ar[0:3] = np.random.normal(0, hamiltonian_error_var, 3)
ar[9] = np.random.normal(0, hamiltonian_error_var)
ar[6] = np.random.normal(0, hamiltonian_error_var)
mdl_datagen.set_fogi_errorgen_components_array(ar, include_fogv=False, normalized_elem_gens=True)
print('hamiltonian-only MSE with target', mserror(mdl_datagen, target_model))
print('hamiltonian-only agsi with target', avg_gs_infidelity(mdl_datagen, target_model))

# add stochastic noise
ar[3:6] = max_stochastic_error_rate*np.random.rand(3)
ar[7:9] = max_stochastic_error_rate*np.random.rand(2)
ar[10:12] = max_stochastic_error_rate*np.random.rand(2)
mdl_datagen.set_fogi_errorgen_components_array(ar, include_fogv=False, normalized_elem_gens=True)

print('MSE with target', mserror(mdl_datagen, target_model))
print('agi with target', avg_gs_infidelity(mdl_datagen, target_model))

hamiltonian-only MSE with target 0.013944923672360084
hamiltonian-only agsi with target -0.00972702911544579
MSE with target 0.01394719147439365
agi with target -0.00948075559480882


In [18]:
prior_covar = (mserror(mdl_datagen, target_model)/target_model.num_params)*np.eye(target_model.num_params)
ekf = ExtendedKalmanFilter(filter_model, prior_covar)

In [20]:
pfids = std.prep_fiducials()
mfids = std.meas_fiducials()

In [23]:
ekf.select_fiducial(germs[0], 1, mfids, pfids)

0 0


ValueError: setting an array element with a sequence.